In [1]:
# Import packages
import os
import time
import pyspark
import findspark
import numpy as np
import pandas as pd

from pyspark import SparkContext, SparkConf
from pyspark.mllib.regression import LabeledPoint

from pyspark.ml import Pipeline
#from pyspark.ml.feature import *
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.feature import StringIndexer, VectorIndexer, OneHotEncoder, VectorAssembler, IndexToString

from pyspark.ml.linalg import Vectors
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.sql.functions import col, when, explode
from pyspark.sql import SparkSession, SQLContext, Row, HiveContext

In [2]:
findspark.init()

# Creatingt Spark SQL environment
spark =SparkSession\
   .builder\
   .appName("test")\
   .enableHiveSupport().getOrCreate()

sc= spark.sparkContext
sqlContext= SQLContext(sc)

findspark.find()

'C:\\spark\\spark-3.0.1-bin-hadoop2.7'

In [3]:
# spark is an existing SparkSession
df = sqlContext.read.format("csv")\
   .option("header", "true")\
   .load("C:/Users/affiqazrin/Desktop/dataset/Data_FinalProject.csv")

In [4]:
#target (y=deposit)
df.groupBy("y").count().show()

+---+-----+
|  y|count|
+---+-----+
| no|36548|
|yes| 4640|
+---+-----+



In [5]:
# Convert Feature Types
df.createOrReplaceTempView("df")

df = spark.sql("select \
                    cast(age as int) as age, \
                    cast(job as string) as job, \
                    cast(marital as string) as marital, \
                    cast(education as string) as education, \
                    cast(default as string) as default, \
                    cast(housing as string) as housing, \
                    cast(loan as string) as loan, \
                    cast(contact as string) as contact, \
                    cast(day_of_week as string) as day, \
                    cast(month as string) as month, \
                    cast(duration as int) as duration, \
                    cast(campaign as int) as campaign, \
                    cast(pdays as int) as pdays, \
                    cast(previous as int) as previous, \
                    cast(poutcome as string) as poutcome, \
                    cast(y as string) as deposit \
                from df")

# Data Types
df.dtypes
[('age', 'int'),
 ('job', 'string'),
 ('marital', 'string'),
 ('education', 'string'),
 ('default', 'string'),
 ('housing', 'string'),
 ('loan', 'string'),
 ('contact', 'string'),
 ('day', 'string'),
 ('month', 'string'),
 ('duration', 'int'),
 ('campaign', 'int'),
 ('pdays', 'int'),
 ('previous', 'int'),
 ('poutcome', 'string'),
 ('deposit', 'string')]

[('age', 'int'),
 ('job', 'string'),
 ('marital', 'string'),
 ('education', 'string'),
 ('default', 'string'),
 ('housing', 'string'),
 ('loan', 'string'),
 ('contact', 'string'),
 ('day', 'string'),
 ('month', 'string'),
 ('duration', 'int'),
 ('campaign', 'int'),
 ('pdays', 'int'),
 ('previous', 'int'),
 ('poutcome', 'string'),
 ('deposit', 'string')]

In [6]:
#categorical attrib(9)
categorical_cols = ["job",
                    "marital",
                    "education",
                    "default",
                    "housing",
                    "loan",
                    "contact",
                    "day",
                    "month",
                    "poutcome",
                    "deposit"]
 
#numerical attrib(6)
numeric_cols = ["age",
                "duration",
                "campaign",
                "pdays",
                "previous"]

In [7]:
# dropping null values
df = df.dropna()

In [8]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
jobIndexer = StringIndexer(inputCol='job', outputCol="indexedJob")
maritalIndexer = StringIndexer(inputCol='marital', outputCol="indexedMarital")
educationIndexer = StringIndexer(inputCol='education', outputCol="indexedEducation")

housingIndexer = StringIndexer(inputCol='housing', outputCol="indexedHousing")
loanIndexer = StringIndexer(inputCol='loan', outputCol="indexedLoan")
contactIndexer = StringIndexer(inputCol='contact', outputCol="indexedContact")

dayIndexer = StringIndexer(inputCol='day', outputCol="indexedDay")
monthIndexer = StringIndexer(inputCol='month', outputCol="indexedMonth")
poutcomeIndexer = StringIndexer(inputCol='poutcome', outputCol="indexedPoutcome")

depositIndexer = StringIndexer(inputCol='deposit', outputCol="indexedDeposit")

In [9]:
# One Hot Encoder on indexed features
jobEncoder = OneHotEncoder(inputCol='indexedJob', outputCol="jobVec").setDropLast(False)
maritalEncoder = OneHotEncoder(inputCol='indexedMarital', outputCol="maritalVec").setDropLast(False)
educationEncoder = OneHotEncoder(inputCol='indexedEducation', outputCol="educationVec").setDropLast(False)

housingEncoder = OneHotEncoder(inputCol='indexedHousing', outputCol="housingVec").setDropLast(False)
loanEncoder = OneHotEncoder(inputCol='indexedLoan', outputCol="loanVec").setDropLast(False)
contactEncoder = OneHotEncoder(inputCol='indexedContact', outputCol="contactVec").setDropLast(False)

dayEncoder = OneHotEncoder(inputCol='indexedDay', outputCol="dayVec").setDropLast(False)
monthEncoder = OneHotEncoder(inputCol='indexedMonth', outputCol="monthVec").setDropLast(False)
poutcomeEncoder = OneHotEncoder(inputCol='indexedPoutcome', outputCol="poutcomeVec").setDropLast(False)

In [10]:
assembler = VectorAssembler(inputCols=['age',
                                       'duration',
                                       'campaign',
                                       'pdays',
                                       'previous',
                                       "jobVec",
                                       "maritalVec",
                                       "educationVec",
                                       "housingVec",
                                       "loanVec",
                                       "contactVec",
                                       "dayVec",
                                       "monthVec",
                                       "poutcomeVec",],outputCol="features")

In [11]:
# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[jobIndexer,
                            maritalIndexer,
                            educationIndexer,
                            housingIndexer,
                            loanIndexer,
                            contactIndexer,
                            dayIndexer,
                            monthIndexer,
                            poutcomeIndexer,
                            depositIndexer,
                            jobEncoder,
                            maritalEncoder,
                            educationEncoder,
                            housingEncoder,
                            loanEncoder,
                            contactEncoder,
                            dayEncoder,
                            monthEncoder,
                            poutcomeEncoder,
                            assembler])
 
# Train model.  This also runs the indexers.
model = pipeline.fit(df)
 
# final spark dataframe
finaldf = model.transform(df)

In [12]:
finaldf.show()

+---+-----------+--------+-------------------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+-----------+-------+----------+--------------+----------------+--------------+-----------+--------------+----------+------------+---------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+--------------+-------------+--------------------+
|age|        job| marital|          education|default|housing|loan|  contact|day|month|duration|campaign|pdays|previous|   poutcome|deposit|indexedJob|indexedMarital|indexedEducation|indexedHousing|indexedLoan|indexedContact|indexedDay|indexedMonth|indexedPoutcome|indexedDeposit|        jobVec|   maritalVec| educationVec|   housingVec|      loanVec|   contactVec|       dayVec|      monthVec|  poutcomeVec|            features|
+---+-----------+--------+-------------------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+-----------+-------+

In [13]:
# Select example rows to display.
finaldf.columns

# Select example rows to display.
finaldf.select("deposit",
               "features").show()

+-------+--------------------+
|deposit|            features|
+-------+--------------------+
|     no|(55,[0,1,2,3,13,1...|
|     no|(55,[0,1,2,3,8,17...|
|     no|(55,[0,1,2,3,8,17...|
|     no|(55,[0,1,2,3,5,17...|
|     no|(55,[0,1,2,3,8,17...|
|     no|(55,[0,1,2,3,8,17...|
|     no|(55,[0,1,2,3,5,17...|
|     no|(55,[0,1,2,3,6,17...|
|     no|(55,[0,1,2,3,7,18...|
|     no|(55,[0,1,2,3,8,18...|
|     no|(55,[0,1,2,3,6,17...|
|     no|(55,[0,1,2,3,8,18...|
|     no|(55,[0,1,2,3,6,18...|
|     no|(55,[0,1,2,3,13,1...|
|     no|(55,[0,1,2,3,6,17...|
|     no|(55,[0,1,2,3,10,1...|
|     no|(55,[0,1,2,3,6,17...|
|     no|(55,[0,1,2,3,6,17...|
|     no|(55,[0,1,2,3,6,17...|
|     no|(55,[0,1,2,3,9,18...|
+-------+--------------------+
only showing top 20 rows



In [14]:
# Spliting in train and test set. Beware : It sorts the dataset
(traindf, testdf) = finaldf.randomSplit([0.7,0.3])

In [ ]:
finaldf.toPandas().to_csv('Data_FinalProject_READY3.csv')

In [15]:
finaldf.show()

+---+-----------+--------+-------------------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+-----------+-------+----------+--------------+----------------+--------------+-----------+--------------+----------+------------+---------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+--------------+-------------+--------------------+
|age|        job| marital|          education|default|housing|loan|  contact|day|month|duration|campaign|pdays|previous|   poutcome|deposit|indexedJob|indexedMarital|indexedEducation|indexedHousing|indexedLoan|indexedContact|indexedDay|indexedMonth|indexedPoutcome|indexedDeposit|        jobVec|   maritalVec| educationVec|   housingVec|      loanVec|   contactVec|       dayVec|      monthVec|  poutcomeVec|            features|
+---+-----------+--------+-------------------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+-----------+-------+